In [2]:
from groq import Groq
import os
import re
import pandas as pd
import json
from langchain.chat_models import init_chat_model

In [ ]:
client = Groq(
    api_key="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"
)
for i in os.listdir("Corrected"):
    with open(f'Corrected/{i}', 'r' , encoding="utf8") as resume_file:
        c=10000
        resume_f = resume_file.read()
        for j in range(c , len(resume_f) - 20000 , 5500  ):
            resume = resume_f[j : j+5000]
            completion = client.chat.completions.create(
                model="llama3-70b-8192",
                messages=[
                    {
                        "role": "user",
                        "content": """ Please extract the numeric values for the following parameters from the provided text:
                                    pH,Calcaire Total,Matière Organique (MO),Capacité d'Échange Cationique (CEC),Conductivité Électrique (CE),Densité Apparente (Da)
                                    Additionally, include the study location, which corresponds to a wilaya in Algeria, based on the provided latitude and longitude in the text, as well as the depth of the soil.
                                    Return the results in JSON format, structured as follows:
                                    {
                                        "location": {
                                            "wilaya": "Name of Wilaya",
                                            "latitude": "latitude_value",
                                            "longitude": "longitude_value"
                                        },
                                        "parameters": {
                                            "pH": numeric_value,
                                            "Calcaire Total": numeric_value,
                                            "Matière Organique (MO)": numeric_value,
                                            "Capacité d'Échange Cationique (CEC)": numeric_value,
                                            "Conductivité Électrique (CE)": numeric_value,
                                            "Densité Apparente (Da)": numeric_value
                                        },
                                        "depth": "depth_value"
                                    }
                                here is the resume
                                """+resume
                    },
                    {
                        "role": "assistant",
                        "content": "return a json response containe the needed data from the resume"
                    }
                ],
                temperature=1,
                max_tokens=1024,
                top_p=1,
                stream=False,
                stop=None,
            )

            output_file_name = f"f.md"    
            with open(output_file_name, 'a') as output_file:
                try : 
                    matches = re.findall(r'```(.*?)```', completion.choices[0].message.content, re.DOTALL)
                    output_file.write(f"this output is for the doc : {i} here is his output \n  {matches[0]}")
                except Exception as e : 
                    pass
        print(f"doc {i} is done")

transforming model responses to dataframe

1. cleaned json version

In [58]:
def remove_comments(json_text):
    cleaned_text = re.sub(r'//.*', '', json_text)
    return cleaned_text
file_path = 'final.json'

with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()
cleaned_content = remove_comments(content)
with open(file_path, 'w', encoding='utf-8') as file:
    content = file.write(cleaned_content)


2. transforme to csv

In [59]:
with open('final.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

rows = []
for index , i in enumerate(data):
    try : 
        row = {
            "Wilaya": i["location"]["wilaya"],
            "Lat": i["location"]["latitude"],
            "Lon": i["location"]["longitude"],
            "Ph": i["parameters"]["pH"],
            "Calcaire Total": i["parameters"]["Calcaire Total"],
            "Matiere Organique (MO)": i["parameters"]["Matiere Organique (MO)"],
            "Capacite d'echange Cationique (CEC)": i["parameters"]["Capacite d'echange Cationique (CEC)"],
            "Conductivite electrique (CE)": i["parameters"]["Conductivite electrique (CE)"],
            "Densite Apparente (Da)": i["parameters"]["Densite Apparente (Da)"],
            "Depth": i["depth"]
        }
        rows.append(row)
    except Exception as e : 
        print(f"in position : {index} ,{e}")
        print(i)

df = pd.DataFrame(rows)

in position : 194 ,'pH'
{'location': {'wilaya': 'Ghardaea', 'latitude': '32.4833', 'longitude': '3.6833'}, 'parameters': {}, 'depth': ''}
in position : 252 ,'pH'
{'location': {'wilaya': 'Biskra', 'latitude': '34.8500e N', 'longitude': '5.7333e E'}, 'parameters': {'Conductivite electrique (CE)': [], 'SAR': {"M'ziraa": 3.98, 'Doucen': 2.38, 'Tolga': 5.56, 'Zeribet El Oued': 5.86, 'El Outaya': 5.1, 'El Hadjeb': 8.16}}, 'depth': []}
in position : 282 ,'pH'
{'location': {'wilaya': 'Not available', 'latitude': 'Not available', 'longitude': 'Not available'}, 'parameters': {}, 'depth': 'Not available'}
in position : 292 ,'pH'
{'location': {'wilaya': 'Not available', 'latitude': 'Not available', 'longitude': 'Not available'}, 'parameters': {'Not available': 'Not available'}, 'depth': 'Not available'}
in position : 395 ,list indices must be integers or slices, not str
[{'location': {'wilaya': 'Batna', 'latitude': '35 34 58', 'longitude': ''}, 'parameters': {'pH': '', 'Calcaire Total': '', 'Matie

In [60]:
df

,Wilaya,Lat,Lon,Ph,Calcaire Total,Matiere Organique (MO),Capacite d'echange Cationique (CEC),Conductivite electrique (CE),Densite Apparente (Da),Depth
0,Jijel,None,None,None,None,None,None,None,None,None
1,Jijel,36.7833,5.7667,7.8,14.15,2.53,24.58,0.25,1.45,0-20 cm
2,,,,None,None,None,None,None,None,
3,NOT_FOUND,NOT_FOUND,NOT_FOUND,NOT_FOUND,NOT_FOUND,NOT_FOUND,NOT_FOUND,NOT_FOUND,NOT_FOUND,NOT_FOUND
4,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
1142,El-Hodna,34.8833e N,5.7167e E,7.83,18.25,1.15,15.32,1.21,1.48,0-90 cm and 0-150 cm
1143,"(latitude: , longitude: )",,,None,None,None,None,None,None,"0-90cm, 0-150cm"
1144,,,,,,,,,,
1145,Tiaret,35.3843e N,1.7947e E,8.52,2.48,21.99,2.47,2.21,1.71,not provided


3. cleaning the dataset column per column

In [61]:
values =[]
def costum_value_counts(column):
    for i in column :
        if values.__contains__(i):
            pass
        else : 
            values.append(i)
    return values 
costum_value_counts(df["Wilaya"])

['Jijel',
 '',
 'NOT_FOUND',
 ' (no wilaya information provided)',
 'Tizi-Ouzou',
 'Annaba',
 'Biskra',
 "M'sila",
 'Tiaret',
 'Baraki',
 'Tizi Ouzou',
 'Medea',
 'Bejaea',
 'Wilaya de Setif',
 None,
 'Pas de Calais',
 'Tipaza',
 'Tebessa',
 'Tlemcen',
 'Mostaganem',
 'Not specified (no location data available)',
 'Wilaya of Tizi Ouzou',
 'Batna',
 'Non specified',
 'Bouira',
 ' Ain Temouchent',
 'Alger',
 'Not specified  (Please provide the location information)',
 'Guelma',
 'Name of Wilaya',
 'Constantine',
 ' ( latitude and longitude',
 'Blida',
 'Not mentioned',
 'Benslimane',
 'Not specified in the provided text',
 'Ain Temouchent',
 'Relizane',
 'Non Specified',
 'Djelfa',
 'Setif',
 'Guerif',
 'Ghardaia',
 ' Ain Defla',
 'Sidi Bel Abbes',
 'Reghaea',
 'El Bayadh',
 'Algiers',
 'Rouiba',
 'Mitidja',
 'Not available',
 ['Metidja',
  'Alger',
  'Blida',
  'Tizi-Ouzou',
  'Azazga',
  'Annaba',
  'Constantine',
  'Oran',
  'Tlemcen',
  'Ain-Temouchent',
  'Beni-Saf',
  'Oasis',
  'L

In [62]:
import numpy as np 
valid_wilayas = [
    'Jijel', 'Tizi-Ouzou', 'Annaba', 'Biskra', "M'sila", 'Tiaret', 'Baraki', 
    'Medea', 'Bejaea', 'Wilaya de Setif', 'Pas de Calais', 'Tipaza', 'Tebessa', 
    'Tlemcen', 'Mostaganem', 'Wilaya of Tizi Ouzou', 'Batna', 'Bouira', 
    'Ain Temouchent', 'Alger', 'Guelma', 'Constantine', 'Blida', 'Ain Temouchent', 
    'Relizane', 'Djelfa', 'Setif', 'Guerif', 'Ghardaia', 'Ain Defla', 
    'Sidi Bel Abbes', 'Algiers', 'Mitidja', 'Bechar', 'Bordj Bou Arreridj', 
    'Chlef', 'Tamanrasset', 'Skikda', 'BATNA', 'Wilaya of Mostaganem', 
    'Gafsa', 'Fes', 'Kerman', 'Safi', 'Oran', 'Constantine', 
    'Wilaya of Constantine', 'Wilaya de Constantine', 'Berriche', 
    'Wilaya de Kais', 'El-Hodna', 'Tindouf', 'Touggourt', 'Oum El Bouaghi', 
    'Wilaya of Mascara', 'Wilaya of Tiaret'
]
df['Wilaya'] = df['Wilaya'].where(df['Wilaya'].isin(valid_wilayas), np.nan)

In [63]:
df

,Wilaya,Lat,Lon,Ph,Calcaire Total,Matiere Organique (MO),Capacite d'echange Cationique (CEC),Conductivite electrique (CE),Densite Apparente (Da),Depth
0,Jijel,None,None,None,None,None,None,None,None,None
1,Jijel,36.7833,5.7667,7.8,14.15,2.53,24.58,0.25,1.45,0-20 cm
2,NaN,,,None,None,None,None,None,None,
3,NaN,NOT_FOUND,NOT_FOUND,NOT_FOUND,NOT_FOUND,NOT_FOUND,NOT_FOUND,NOT_FOUND,NOT_FOUND,NOT_FOUND
4,NaN,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
1142,El-Hodna,34.8833e N,5.7167e E,7.83,18.25,1.15,15.32,1.21,1.48,0-90 cm and 0-150 cm
1143,NaN,,,None,None,None,None,None,None,"0-90cm, 0-150cm"
1144,NaN,,,,,,,,,
1145,Tiaret,35.3843e N,1.7947e E,8.52,2.48,21.99,2.47,2.21,1.71,not provided


In [64]:
df["Wilaya"].isna().sum()

np.int64(473)

We gonna replace this values by their wilaya after cleaning the lat and lon columns

In [65]:
def clean_lat(value):
    if value =="" or value ==None:
        return np.nan
    try:
        new_value = float(str(value).replace("e" , "").replace("N","").replace("," , "."))
    except Exception as e :

        print(e)
        return np.nan
    return new_value

df["Lat"] = df["Lat"].apply(clean_lat)
df

could not convert string to float: 'OT_FOUD'
could not convert string to float: ' (no latitud information providd)'
could not convert string to float: 'ot spcifid (no location data availabl)'
could not convert string to float: 'on spcifid'
could not convert string to float: 'ot spcifid  (Plas provid th location information)'
could not convert string to float: 'latitud_valu'
could not convert string to float: '30 43 '
could not convert string to float: 'ot mntiond'
could not convert string to float: 'ot spcifid in th providd txt'
could not convert string to float: 'on Spcifid'
could not convert string to float: 'ot mntiond'
could not convert string to float: '36.42.319 '
could not convert string to float: 'otfound'
could not convert string to float: 'ot availabl'
could not convert string to float: 'ot Found'
could not convert string to float: 'ot found '
could not convert string to float: 'ot availabl'
could not convert string to float: 'ot availabl'
could not convert string to float: '

,Wilaya,Lat,Lon,Ph,Calcaire Total,Matiere Organique (MO),Capacite d'echange Cationique (CEC),Conductivite electrique (CE),Densite Apparente (Da),Depth
0,Jijel,NaN,None,None,None,None,None,None,None,None
1,Jijel,36.7833,5.7667,7.8,14.15,2.53,24.58,0.25,1.45,0-20 cm
2,NaN,NaN,,None,None,None,None,None,None,
3,NaN,NaN,NOT_FOUND,NOT_FOUND,NOT_FOUND,NOT_FOUND,NOT_FOUND,NOT_FOUND,NOT_FOUND,NOT_FOUND
4,NaN,NaN,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
1142,El-Hodna,34.8833,5.7167e E,7.83,18.25,1.15,15.32,1.21,1.48,0-90 cm and 0-150 cm
1143,NaN,NaN,,None,None,None,None,None,None,"0-90cm, 0-150cm"
1144,NaN,NaN,,,,,,,,
1145,Tiaret,35.3843,1.7947e E,8.52,2.48,21.99,2.47,2.21,1.71,not provided


In [66]:
df["Lat"].isna().sum()

np.int64(433)

In [67]:
def clean_lon(value):
    if value =="" or value ==None:
        return np.nan
    try:
        new_value = float(str(value).replace("e" , "").replace("E","").replace("," , "."))
    except Exception as e :

        print(e)
        return np.nan
    return new_value
df["Lon"] = df["Lon"].apply(clean_lon)
df

could not convert string to float: 'NOT_FOUND'
could not convert string to float: ' (no longitud information providd)'
could not convert string to float: 'Not spcifid (no location data availabl)'
could not convert string to float: 'Non spcifid'
could not convert string to float: 'Not spcifid  (Plas provid th location information)'
could not convert string to float: 'longitud_valu'
could not convert string to float: '2 14 '
could not convert string to float: 'Not mntiond'
could not convert string to float: 'Not spcifid in th providd txt'
could not convert string to float: 'Non Spcifid'
could not convert string to float: 'Not mntiond'
could not convert string to float: '3.9.573 '
could not convert string to float: 'Notfound'
could not convert string to float: 'Not availabl'
could not convert string to float: 'Not Found'
could not convert string to float: 'Not found '
could not convert string to float: 'Not availabl'
could not convert string to float: 'Not availabl'
could not convert stri

,Wilaya,Lat,Lon,Ph,Calcaire Total,Matiere Organique (MO),Capacite d'echange Cationique (CEC),Conductivite electrique (CE),Densite Apparente (Da),Depth
0,Jijel,NaN,NaN,None,None,None,None,None,None,None
1,Jijel,36.7833,5.7667,7.8,14.15,2.53,24.58,0.25,1.45,0-20 cm
2,NaN,NaN,NaN,None,None,None,None,None,None,
3,NaN,NaN,NaN,NOT_FOUND,NOT_FOUND,NOT_FOUND,NOT_FOUND,NOT_FOUND,NOT_FOUND,NOT_FOUND
4,NaN,NaN,NaN,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
1142,El-Hodna,34.8833,5.7167,7.83,18.25,1.15,15.32,1.21,1.48,0-90 cm and 0-150 cm
1143,NaN,NaN,NaN,None,None,None,None,None,None,"0-90cm, 0-150cm"
1144,NaN,NaN,NaN,,,,,,,
1145,Tiaret,35.3843,1.7947,8.52,2.48,21.99,2.47,2.21,1.71,not provided


In [284]:
df_copy = df.dropna(subset=['Wilaya', 'Lat', 'Lon'], how='all')
df_copy

,Wilaya,Lat,Lon,Ph,Calcaire Total,Matiere Organique (MO),Capacite d'echange Cationique (CEC),Conductivite electrique (CE),Densite Apparente (Da),Depth
0,Jijel,NaN,NaN,None,None,None,None,None,None,None
1,Jijel,36.783300,5.766700,7.8,14.15,2.53,24.58,0.25,1.45,0-20 cm
5,Jijel,NaN,NaN,None,None,None,None,None,None,None
6,Jijel,36.783300,5.733300,7.8,24,12.6,21,1.7,1.3,20-30 cm
7,Jijel,36.783300,5.750000,7.6,9.2,70.2,15.4,340,1.25,0-20cm
...,...,...,...,...,...,...,...,...,...,...
1138,NaN,31.950000,5.330000,None,None,None,None,5.32,None,1
1139,Tiaret,35.383333,2.433333,8.2,13.8,1.1,22.5,1.19,1.42,0-150 cm
1140,Tlemcen,34.885300,NaN,None,None,None,None,50,None,not specified
1142,El-Hodna,34.883300,5.716700,7.83,18.25,1.15,15.32,1.21,1.48,0-90 cm and 0-150 cm


In [285]:
df_copy.to_csv("output.csv")

In [286]:
def clean_all(value):
    try : 
        if value == None or value == "":
            return np.nan
        
        elif isinstance(value , float) or isinstance(value , int): 
            return float(value)
        
        value = value.replace("dS/m" , "").replace(" and " ,"-").replace(" to " ,"-")
        if len(value.split("-")) >1:
            return [float(value.split("-")[0]) , float(value.split("-")[1])]
        
        else : 
            return np.nan
            
    except Exception as e:
        print(e)
df_copy["Ph"] = df_copy["Ph"].apply(clean_all)
df_copy

'dict' object has no attribute 'replace'
'dict' object has no attribute 'replace'
'dict' object has no attribute 'replace'
'dict' object has no attribute 'replace'
'list' object has no attribute 'replace'
'list' object has no attribute 'replace'
could not convert string to float: ''
'list' object has no attribute 'replace'


C:\Users\abdo7\AppData\Local\Temp\ipykernel_8608\1006459228.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy["Ph"] = df_copy["Ph"].apply(clean_all)


,Wilaya,Lat,Lon,Ph,Calcaire Total,Matiere Organique (MO),Capacite d'echange Cationique (CEC),Conductivite electrique (CE),Densite Apparente (Da),Depth
0,Jijel,NaN,NaN,NaN,None,None,None,None,None,None
1,Jijel,36.783300,5.766700,7.8,14.15,2.53,24.58,0.25,1.45,0-20 cm
5,Jijel,NaN,NaN,NaN,None,None,None,None,None,None
6,Jijel,36.783300,5.733300,7.8,24,12.6,21,1.7,1.3,20-30 cm
7,Jijel,36.783300,5.750000,7.6,9.2,70.2,15.4,340,1.25,0-20cm
...,...,...,...,...,...,...,...,...,...,...
1138,NaN,31.950000,5.330000,NaN,None,None,None,5.32,None,1
1139,Tiaret,35.383333,2.433333,8.2,13.8,1.1,22.5,1.19,1.42,0-150 cm
1140,Tlemcen,34.885300,NaN,NaN,None,None,None,50,None,not specified
1142,El-Hodna,34.883300,5.716700,7.83,18.25,1.15,15.32,1.21,1.48,0-90 cm and 0-150 cm


In [287]:
df_copy["Ph"].isna().sum()

np.int64(229)

In [288]:
import numpy as np

def clean_depth(value):
    try:
        if value is None or value == "":
            return np.nan
        
        if isinstance(value, (float, int)):
            return value
        
        if isinstance(value, (list, dict)):
            return np.nan
        
        value = value.replace("and", "-").replace("to", "-").replace("accumulation calcaire", "-").replace("<", "").replace(">", "").replace("entre", "0")
        if "," in value:
            ranges = value.split(",")
            cleaned_ranges = []
            for r in ranges:
                r = r.strip() 
                if "cm" in r:
                    r = r.replace("cm", "").strip()
                    cleaned_ranges.extend([float(val) for val in r.split("-")])
                elif "m" in r:
                    r = r.replace("m", "").strip()
                    cleaned_ranges.extend([float(val) * 100 for val in r.split("-")])
                else:
                    cleaned_ranges.extend([float(val) for val in r.split("-")])
            return cleaned_ranges
        
        if "-" in value:
            if "cm" in value:
                value = value.replace("cm", "")
                return [float(val.strip()) for val in value.split("-")]
            elif "m" in value:
                value = value.replace("m", "")
                return [float(val.strip()) * 100 for val in value.split("-")]
            else:
                return [float(val.strip()) for val in value.split("-")]
        
        value = re.sub(r'^[^\d]*(\d+)', r'\1', value)  # keep only the first numeric part
        if "cm" in value:
            return float(value.replace("cm", "").strip())
        elif "m" in value or "metre" in value:
            return float(value.replace("metre", "").replace("m", "").strip()) * 100
        
        return float(value)

    except Exception as e:
        try : 
            value = value.replace("e" , "-")
            if "cm" in value:
                value = value.replace("cm", "")
                return [float(val.strip()) for val in value.split("-")]
            elif "m" in value:
                value = value.replace("m", "")
                return [float(val.strip()) * 100 for val in value.split("-")]
            else:
                return [float(val.strip()) for val in value.split("-")]
            
        except Exception as e : 
            print(f"error fel {value} : {e}")

        return np.nan


In [289]:
for i in df.columns[4:]:
    print(i)
    if i == "Depth":
        df_copy[i] = df_copy[i].apply(clean_depth)
    else : 
        df_copy[i] = df_copy[i].apply(clean_all)
df_copy

Calcaire Total
'dict' object has no attribute 'replace'
'dict' object has no attribute 'replace'
'dict' object has no attribute 'replace'
'dict' object has no attribute 'replace'
could not convert string to float: '16% '
'dict' object has no attribute 'replace'
'list' object has no attribute 'replace'
could not convert string to float: ''
'list' object has no attribute 'replace'
Matiere Organique (MO)
'dict' object has no attribute 'replace'
'dict' object has no attribute 'replace'
'dict' object has no attribute 'replace'
'dict' object has no attribute 'replace'
could not convert string to float: ''
'list' object has no attribute 'replace'
Capacite d'echange Cationique (CEC)
'dict' object has no attribute 'replace'
'dict' object has no attribute 'replace'
'dict' object has no attribute 'replace'
'dict' object has no attribute 'replace'
'list' object has no attribute 'replace'
'dict' object has no attribute 'replace'
'list' object has no attribute 'replace'
Conductivite electrique (CE)


C:\Users\abdo7\AppData\Local\Temp\ipykernel_8608\3417759416.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy[i] = df_copy[i].apply(clean_all)
C:\Users\abdo7\AppData\Local\Temp\ipykernel_8608\3417759416.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy[i] = df_copy[i].apply(clean_depth)


,Wilaya,Lat,Lon,Ph,Calcaire Total,Matiere Organique (MO),Capacite d'echange Cationique (CEC),Conductivite electrique (CE),Densite Apparente (Da),Depth
0,Jijel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Jijel,36.783300,5.766700,7.8,14.15,2.53,24.58,0.25,1.45,"[0.0, 20.0]"
5,Jijel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Jijel,36.783300,5.733300,7.8,24.0,12.6,21.0,1.7,1.3,"[20.0, 30.0]"
7,Jijel,36.783300,5.750000,7.6,9.2,70.2,15.4,340.0,1.25,"[0.0, 20.0]"
...,...,...,...,...,...,...,...,...,...,...
1138,NaN,31.950000,5.330000,NaN,NaN,NaN,NaN,5.32,NaN,1.0
1139,Tiaret,35.383333,2.433333,8.2,13.8,1.1,22.5,1.19,1.42,"[0.0, 150.0]"
1140,Tlemcen,34.885300,NaN,NaN,NaN,NaN,NaN,50.0,NaN,NaN
1142,El-Hodna,34.883300,5.716700,7.83,18.25,1.15,15.32,1.21,1.48,"[0.0, 90.0, 0.0, 150.0]"


In [292]:
for i  in df_copy.columns:
    print(df_copy[i].isna().sum())

197
157
184
229
266
256
283
264
324
255


In [294]:
df_copy = df_copy.dropna()
df_copy

,Wilaya,Lat,Lon,Ph,Calcaire Total,Matiere Organique (MO),Capacite d'echange Cationique (CEC),Conductivite electrique (CE),Densite Apparente (Da),Depth
1,Jijel,36.783300,5.766700,7.8,14.15,2.53,24.58,0.25,1.45,"[0.0, 20.0]"
6,Jijel,36.783300,5.733300,7.8,24.0,12.6,21.0,1.7,1.3,"[20.0, 30.0]"
7,Jijel,36.783300,5.750000,7.6,9.2,70.2,15.4,340.0,1.25,"[0.0, 20.0]"
9,Tizi-Ouzou,36.716700,4.050000,8.21,25.0,2.5,15.0,0.25,1.6,"[0.0, 20.0]"
11,Biskra,34.850000,5.730000,8.1,24.51,1.42,24.51,0.41,1.42,"[0.0, 20.0]"
...,...,...,...,...,...,...,...,...,...,...
1119,Tiaret,35.383300,1.850000,8.1,23.4,1.4,15.2,2.5,1.6,"[0.0, 20.0]"
1124,Mostaganem,35.933300,0.133300,8.2,143.5,1.23,15.6,124.5,1.45,150.0
1130,Constantine,36.365100,6.612700,8.55,87.24,5.19,22.21,49.0,1.34,"[0.0, 30.0]"
1139,Tiaret,35.383333,2.433333,8.2,13.8,1.1,22.5,1.19,1.42,"[0.0, 150.0]"


In [295]:
df_copy.to_csv("clean.csv")